In [1]:
import pandas as pd, numpy as np

In [2]:
seed = 1
np.random.seed(seed)

In [3]:
%matplotlib inline

## Load data

In [4]:
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
train_df.shape, test_df.shape

((4209, 378), (4209, 377))

In [5]:
train_df.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [6]:
test_df.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


# preprocessing

In [5]:
from sklearn.preprocessing import LabelEncoder

In [7]:
# train_input = train_df.drop("ID") First, experiment with keeping id
train_input = train_df.copy()
test_input = test_df.copy()

In [8]:
num_train =len( train_input)
num_train

4209

In [9]:
full_input = pd.concat([train_input, test_input])
full_input.shape

(8418, 378)

In [10]:
categorical_cols = full_input.select_dtypes(include=[np.object]).columns.tolist()
categorical_cols

['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']

In [11]:
full_input = pd.get_dummies(full_input)

In [12]:
full_input.drop('y', inplace=True, axis=1)

In [31]:
drop_id = True # Experiment with this value.

In [32]:
if drop_id:
    full_input.drop('ID', inplace=True, axis=1)

In [33]:
full_input_x = full_input[:-1]
full_input_x.shape

(8417, 579)

In [34]:
full_input_y = full_input.shift(-1)[:-1]
full_input_y.shape

(8417, 579)

In [35]:
input_output_dim = full_input_y.shape[1]
input_output_dim

579

## model

In [36]:
from sklearn.metrics import r2_score

In [40]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers.normalization import BatchNormalization

In [43]:
model = Sequential()
model.add(Dense(input_output_dim, input_dim=input_output_dim, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(input_output_dim, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [44]:
model.fit(full_input_x.values, full_input_y.values, epochs=100, shuffle=True, validation_split=0.2, verbose=2)

Train on 6733 samples, validate on 1684 samples
Epoch 1/100
14s - loss: 0.4010 - acc: 0.8359 - val_loss: 0.1804 - val_acc: 0.9337
Epoch 2/100
11s - loss: 0.1732 - acc: 0.9339 - val_loss: 0.1744 - val_acc: 0.9339
Epoch 3/100
10s - loss: 0.1705 - acc: 0.9341 - val_loss: 0.1745 - val_acc: 0.9336
Epoch 4/100


KeyboardInterrupt: 

## Kaggle predictions

In [77]:
kaggle_preds = gbm.predict(test_input.values)
kaggle_preds.shape

(4209,)

In [79]:
test_df['y'] = kaggle_preds
test_df[['ID', 'y']].head()

,ID,y
0,1,79.069446
1,2,95.031323
2,3,78.065370
3,4,78.209953
4,5,115.343554


In [80]:
test_df[['ID', 'y']].to_csv("../data/submission_lightgbm_benchmark.csv", index=False)